# Видео для вдохновения
1. https://www.youtube.com/watch?v=_7vFY3JUhn0
2. https://www.youtube.com/watch?v=Y0m4hVQztD4

In [53]:
import pandas as pd
import numpy as np
import re
import math

# Парсим файл

In [54]:
# Считаем файл

with open('DAV_4.txt', 'r') as file:
    lines = file.read().split('\n')

In [55]:
lines

['max',
 '4x_1+5x_2',
 '2x_1+4x_2 <= 560',
 '1x_1+1x_2 <= 170',
 '2x_1+1x_2 <= 300']

In [56]:
# Функция для парсинга выражения
def str_to_arr(str):
  drop_smb = re.sub(r'x_\d+', '', str)
  tokens = re.findall(r'[+-]?\d+', drop_smb)
  numbers = [int(token) for token in tokens]
  arr = np.array(numbers)
  return arr

In [57]:
str_to_arr(lines[1])

array([4, 5])

In [58]:
# Берем первую строку (ЦФ)
target = str_to_arr(lines[1])
if lines[0] == 'min':
    target = -target

vars = len(target)

In [59]:
# Из ограничений делаем функции (массив и ограничение)
add_vars = 0
def restr_to_func(restr):
  splitted = restr.split(' ')
  rs = float(splitted[-1])
  restr = str_to_arr(splitted[0])
  for i in range(add_vars):
    restr = np.append(restr, 0)

  return restr, rs


all_restr = []
all_rs = []
for i in range(len(lines) - 2):
    restr = lines[i+2]
    if '>=' in restr:
        restr, rs = restr_to_func(restr)
        all_rs.append(rs)
        restr = np.append(restr, -1)
        add_vars += 1
    elif '<=' in restr:
        restr, rs = restr_to_func(restr)
        all_rs.append(rs)
        restr = np.append(restr, +1)
        add_vars += 1
    else:
        restr = restr_to_func(restr)
    all_restr.append(restr)

In [60]:
all_restr

[array([2, 4, 1]), array([1, 1, 0, 1]), array([2, 1, 0, 0, 1])]

In [61]:
all_rs

[560.0, 170.0, 300.0]

In [62]:
# Создаем матрицу из ограничений
# первый элемент
matrix = np.append(all_restr[0], np.zeros(vars + add_vars - len(all_restr[0])))
# Остальные циклом
for i in range(len(all_restr) - 1):
  with_zeroes = np.append(all_restr[i+1], np.zeros(vars + add_vars - len(all_restr[i+1])))
  matrix = np.vstack((matrix, with_zeroes))
matrix

# Добавляем 0 дополнительных переменных в ЦФ
target = np.append(target, np.zeros(add_vars))

In [63]:
# Наши итоговые данные

c = target.tolist()
A = matrix.tolist()
b = all_rs

print(c)
print(A)
print(b)
print(vars)

[4.0, 5.0, 0.0, 0.0, 0.0]
[[2.0, 4.0, 1.0, 0.0, 0.0], [1.0, 1.0, 0.0, 1.0, 0.0], [2.0, 1.0, 0.0, 0.0, 1.0]]
[560.0, 170.0, 300.0]
2


# Алгоритм симплекс-метода

In [64]:
# Матрица данных, с которыми будем работать

xb = [eq + [x] for eq, x in zip(A, b)]
z = c + [0]
data = xb+[z]

data

[[2.0, 4.0, 1.0, 0.0, 0.0, 560.0],
 [1.0, 1.0, 0.0, 1.0, 0.0, 170.0],
 [2.0, 1.0, 0.0, 0.0, 1.0, 300.0],
 [4.0, 5.0, 0.0, 0.0, 0.0, 0]]

In [65]:
# Функция выбора позиции
def get_position(data):
    z = data[-1]
    column = next(i for i, x in enumerate(z[:-1]) if x > 0) # ищем индекс первого положительного элемента
    # column = max((i for i, x in enumerate(z[:-1]) if x > 0), key=lambda i: z[i])

    # Делим все ограничения на элемент от найденного индекса
    rest = []
    for eq in data[:-1]:
        el = eq[column]
        # print(el)
        rest.append(math.inf if el <= 0 else eq[-1] / el)
        # rest.append(eq[-1] / el)
        # print(rest)

    if (all([r == math.inf for r in rest])):
        raise Exception("Linear program is unbounded.")

    row = rest.index(min(rest)) # индекс ряда с новым минимальным элементом
    return row, column

get_position(data)

(2, 0)

In [66]:
def pivot_step(data, pivot_position):
    new_data = [[] for eq in data]

    i, j = pivot_position
    pivot_value = data[i][j] #значение найденной позиции
    # print(pivot_value)
    new_data[i] = np.array(data[i]) / pivot_value #делим старый ряд на значение найденной позиции

    # построчно умнжаем строку старой даты на значение из старой даты, а затем вычитае
    for eq_i, eq in enumerate(data):
        if eq_i != i:
          multiplier = np.array(new_data[i]) * data[eq_i][j]
          new_data[eq_i] = np.array(data[eq_i]) - multiplier

    return new_data

In [67]:
pivot_step(data, (2,0))

[array([  0.,   3.,   1.,   0.,  -1., 260.]),
 array([ 0. ,  0.5,  0. ,  1. , -0.5, 20. ]),
 array([  1. ,   0.5,   0. ,   0. ,   0.5, 150. ]),
 array([   0.,    3.,    0.,    0.,   -2., -600.])]

In [72]:
def get_solution(tab):
    columns = np.array(tab).T
    # print(columns)
    solutions = []
    for column in columns[:-1]:
        solution = 0
        if sum(column) == 1 and len([c for c in column if c == 0]) == len(column) - 1: # базовый ли вектор
            one_index = column.tolist().index(1)
            solution = columns[-1][one_index]
        solutions.append(solution)

    return solutions

get_solution(data)

[0, 0, 560.0, 170.0, 300.0]

In [69]:
tdata = data.copy()
tdata

[[2.0, 4.0, 1.0, 0.0, 0.0, 560.0],
 [1.0, 1.0, 0.0, 1.0, 0.0, 170.0],
 [2.0, 1.0, 0.0, 0.0, 1.0, 300.0],
 [4.0, 5.0, 0.0, 0.0, 0.0, 0]]

In [70]:
tdata = data.copy()
def simplex_method(tdata):
    while any(x > 0 for x in tdata[-1][:-1]):
        pivot_position = get_position(tdata)
        tdata = pivot_step(tdata, pivot_position)

    return get_solution(tdata)

In [71]:
sol = simplex_method(tdata)
np.dot(sol, c)

790.0